This notebook shows a simple method to identify Kollar divisors using CYTools.

Recall, Kollar divisors are those, $D\in K(X)\cap\mathbb{Z}^{h^{1,1}}$, for which $\kappa_{ijk} D^i D^j D^k = 0$ but $\kappa_{ijk} D^i D^j \neq 0$.

# Setup

## Imports

Here, we import some useful packages...

In [ ]:
# generic Python imports
import itertools
import numpy as np

# CYTools imports
from cytools import Cone, fetch_polytopes, Polytope

# custom impotys
import sys; sys.path.append('../cytools_utilities/')
from ntfe_frsts import ntfe_frsts
from gvs import gvs

Filter warnings

In [ ]:
import warnings
warnings.filterwarnings("ignore")

## Custom functions

Here, we define some useful functions...

In [ ]:
def read_wati_polys(inds, fname="fiber-free.txt"):
    """
    Read polytopes from Wati's list.
    
    From http://ctp.lns.mit.edu/wati/data.html
    """
    all_polys = []

    # load the points
    with open(fname, "r") as f:
        for i_poly in range(max(poly_inds)+1):
            # read the points
            pts = []

            f.readline()
            for i_coords in range(4):
                pts.append([int(x) for x in f.readline().split()])
            pts = np.array(pts).T

            # this is a polytope of interest
            if i_poly in poly_inds:
                all_polys.append(Polytope(pts))
    
    return all_polys

In [ ]:
def intersection_form(kappa, x, order=3):
    # get the indices to contract
    inds = 'abc,' + ','.join([chr(ord('a')+i) for i in range(order)])
    
    # contract them!
    return np.einsum(inds, kappa, *(order*[x]))

# The scan!

Select polytopes to scan over

In [ ]:
scan_over_wati = False

In [ ]:
if scan_over_wati:
    # indices of the polytopes to scan over http://ctp.lns.mit.edu/wati/data.html
    poly_inds = list(range(20,600))
    
    # read them from the file
    polys = read_wati_polys(poly_inds)
else:
    # read from KS
    polys = fetch_polytopes(h11=6,lattice="N",limit=50,as_list=True)

Do the scan!

In [ ]:
verbosity = 0

In [ ]:
kollar_divisors = []
found_divisors = 0

for i,p in enumerate(polys):
    # get the Hodge numbers, dualize to have h11<h21
    h11 = p.h11(lattice="N")
    h21 = p.h21(lattice="N")
    if h11>h21:
        p = p.dual()
        
    # reject non-favorables
    if not p.is_favorable(lattice="N"):
        continue

    print(f"poly #{i}, (h11,h21)={(p.h11(lattice='N'),p.h21(lattice='N'))}, total found (with duplicates)={found_divisors}",end='\r')

    kollar_divisors.append([])
    
    # iterate over NTFEs
    # ------------------
    ts = p.ntfe_frsts()
    if verbosity>0:
        print(f"There are {len(ts)} NTFEs")

    for t in ts:
        kollar_divisors[-1].append([])

        # construct the TV, CY
        # --------------------
        cy = t.cy()
        tv = cy.ambient_variety()
        if verbosity>1:
            print('Constructing CY, TV...')

        # get the intersection numbers
        # ----------------------------
        if verbosity>1:
            print('Constructing kappa...')
        kappa = cy.intersection_numbers(format='dense',in_basis=True)

        # get the Kahler cone
        # -------------------
        if verbosity>1:
            print('Constructing Kähler cones...')
        if False:
            # get the CY kahler cone
            print('with GVs...')
            max_deg = 10
            
            while True:
                try:
                    gvs = cy.compute_gvs(max_deg=max_deg)
                    break
                except:
                    max_deg += 2
    
            mori = Cone(rays=list(gvs.keys()))
            kahler = mori.dual()
            kahler = Cone(rays=kahler.extremal_rays())
        else:
            # get the toric kahler cone
            kahler = cy.toric_kahler_cone()

        # find the possible Kollar divisors
        # ---------------------------------
        # (lattice points in the Kahler cone...)
        min_points = 200
        if verbosity>1:
            print(f"Finding >={min_points} points...")
        possible_pts = kahler.find_lattice_points(min_points=min_points, verbose=False)
        if verbosity>1:
            print(f"Found {len(possible_pts)} points...")
        
        # check them!
        # -----------
        for kollar_plz in possible_pts:
            # skip the origin... trivial divisor
            if sum(np.abs(kollar_plz))==0:
                continue

            # check if cubic is 0
            if intersection_form(kappa, kollar_plz, order=3) == 0:
                # get the quadratic
                quadratic = intersection_form(kappa, kollar_plz, order=2)

                # check if the quadratic is nonzero
                if any(quadratic!=0) and all(quadratic>=0):
                    kollar_divisors[-1][-1].append(kollar_plz)
                    found_divisors += 1
                    
                    # extra printing
                    if verbosity>0:
                        print(100*":)")
                        print(f"x={kollar_plz}")
                        print()

        # extra printing
        if verbosity>1:
            print('all done!')
            print()